## Load Functions for Printing Numbers and Matrices

with better formatting. The notebook uses the functions `printmat()` and `printlnPs()`.

In [1]:
using Dates            

include("jlFiles/printmat.jl")

printlnPs (generic function with 2 methods)

# Return Calculations

The return of holding the asset between $t-1$ and $t$ is

$
R_t = (P_t+D_t)/P_{t-1} - 1
$,

where $P_t$ is the price (measured after dividends) and $D_t$ is the dividend.

In [2]:
P = [100;108;109]           #price series, after dividends
D = [0;2;0]                 #dividends
t = 1:length(P)             #time

R = (P[2:end] + D[2:end])./P[1:end-1] .- 1        #notice: .-

println("   period    return, %")
printmat([t[2:end] R*100])

   period    return, %
     2.000    10.000
     3.000     0.926



# Cumulating Returns

Net returns can be cumulated into a portfolio value as 

$
V_t = V_{t-1}(1+R_t)
$

where we need a starting value (initial investment) for the portfolio (a common choice is to normalise to $V_0=1$).

With log returns, $r_t=\log(1+R_t)$, we instead do 

$
\ln V_t = \ln V_{t-1} + r_t
$


In [3]:
t   = 1:3
R   = [20;-35;25]/100                #returns for t=1,2,3
V   = cumprod(1 .+ R)                #V(t) = V(t-1)*(1+R(t)), starting at V(0)=1 
lnV = cumsum(log.(1 .+ R))           #lnV(t) = lnV(t-1) + r(t) 

println("   period    return       V        logV")
printmat([t R V lnV])

   period    return       V        logV
     1.000     0.200     1.200     0.182
     2.000    -0.350     0.780    -0.248
     3.000     0.250     0.975    -0.025



# Portfolio Return: Definition, Expected Value and Variance

With portfolio weights in the vector $w$, the portfolio return, the expected portfolio return and the portfolio variance can be computed as

$R_p = w'R$, 

$\text{E}R_p = w'\mu$ and 

$\text{Var}(R_p) = w'\Sigma w$

In [4]:
w = [0.8;0.2]
R = [10;5]/100          #returns of asset 1 and 2
Rp = w'R

println("portfolio weights: ")
printmat(w)
println("Returns of assets: ")
printmat(R)
println("Portfolio return: ")
printlnPs(Rp)

portfolio weights: 
     0.800
     0.200

Returns of assets: 
     0.100
     0.050

Portfolio return: 
     0.090


In [5]:
μ = [9;6]/100                    #\mu and tab to get this
Σ = [256 96;
     96 144]/100^2

println("expected returns*100: ")
printmat(μ*100)

println("covariance matrix*100^2:")
printmat(Σ*100^2)

expected returns*100: 
     9.000
     6.000

covariance matrix*100^2:
   256.000    96.000
    96.000   144.000



In [6]:
ERp   = w'μ
VarRp = w'Σ*w

println("expected portfolio return, variance and std of portfolio return: ")
printmat([ERp VarRp sqrt(VarRp)])

expected portfolio return, variance and std of portfolio return: 
     0.084     0.020     0.142



In [7]:
Σb = [256 -96;
      -96 144]/100^2
                                        
printlnPs("portfolio std if the assets were negatively correlated: ",sqrt(w'Σb*w))

portfolio std if the assets were negatively correlated:      0.118


# Matrix Appendix

with Julia (and textbook mathematical notation)

With a matrix $A$ and a scalar $c$, do

1. `A*c` (textbook: $Ac$) to multiply each element of $A$ by $c$

2. `A .+ c` (textbook: $A+cJ$, where $J$ is a matrix of ones) to add $c$ to each element of $A$ 

In [8]:
A = [1 3;3 4]                                    
c = 10

println("A:")
printmat(A)
println("c:")
printmat(c)

println("A*c:")
printmat(A*c)

println("A .+ c:")
printmat(A .+ c)

A:
         1         3
         3         4

c:
        10

A*c:
        10        30
        30        40

A .+ c:
        11        13
        13        14



With two matrices of the same dimensions ($A$ and $B$), do

`A+B` (textbook: $A+B$) to add them (element by element)

In [9]:
A = [10;11]                 #A and B are 2x1 matrices
B = [2;5]

println("A:")
printmat(A)
println("B:")
printmat(B)

println("A-B:")
printmat(A-B)

A:
        10
        11

B:
         2
         5

A-B:
         8
         6



In [10]:
A = [1 3;3 4]               #A and B are 2x2 matrices
B = [1 2;3 -2]
println("A:")
printmat(A)
println("B:")
printmat(B)

println("A+B:")
printmat(A+B)

A:
         1         3
         3         4

B:
         1         2
         3        -2

A+B:
         2         5
         6         2



Transpose a matrix as  `A'` (textbook: $A'$)

In [11]:
A = [10;11]
println("A: ")
printmat(A)
println("A': ")
printmat(A')

B = [1 2 3;4 5 6]
println("B: ")
printmat(B)
println("B': ")
printmat(B')

A: 
        10
        11

A': 
        10        11

B: 
         1         2         3
         4         5         6

B': 
         1         4
         2         5
         3         6



Multiplying matrices ($A$ and $B$) of conformable dimensions

`A*B` (textbook: $AB$)

In [12]:
A = [1 3;3 4]           #A and B are 2x2 matrices
B = [1 2;3 -2]
println("A:")
printmat(A)
println("B:")
printmat(B)

println("A*B:")
printmat(A*B)

A:
         1         3
         3         4

B:
         1         2
         3        -2

A*B:
        10        -4
        15        -2



In [13]:
A = [1 3;3 4]           #A is 2x2, B is a vector with 2 elements
B = [2;5]
println("A:")
printmat(A)
println("B:")
printmat(B)

println("A*B:")
printmat(A*B)

A:
         1         3
         3         4

B:
         2
         5

A*B:
        17
        26



Inner and outer products of (column) vectors with $k$ elements:

`x'z` (textbook: $x'z$) to get a scalar and `x*z'` (textbook: $xz'$) to get a $k\times k$ matrix  

In [14]:
x = [10;11]
z = [2;5]
println("x and z")
printmat([x z])

println("x'z: ")
printlnPs(x'z)

println("x*z':")
printmat(x*z')

x and z
        10         2
        11         5

x'z: 
        75
x*z':
        20        50
        22        55



A quadratic form ($A$ is an $n \times n$ matrix and x is an $n$ vector)

`x'A*x` (textbook: $x'Ax$) to get a scalar

In [15]:
A = [1 3;3 4]

println("x:")
printmat(x)
println("A:")
printmat(A)

println("x'A*x: ")
printlnPs(x'A*x)

x:
        10
        11

A:
         1         3
         3         4

x'A*x: 
      1244


A matrix inverse of an $nxn$ matrix $A$:

`inv(A)` or `A^(-1)` (textbook: $A^{-1}$)

The inverse is such that $AA^{-1}=I$ and $A^{-1}A=I$

In [16]:
println("A:")
printmat(A)

println("inv(A):")
printmat(inv(A))

println("inv(A)*A:")
printmat(inv(A)*A)

A:
         1         3
         3         4

inv(A):
    -0.800     0.600
     0.600    -0.200

inv(A)*A:
     1.000    -0.000
     0.000     1.000

